In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import Libraries

import numpy as np
import pandas as pd
import os
import re
import requests
import datetime

from bs4 import BeautifulSoup

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# 10 Categories will be used

categories_dict = {  
    "automotive":{
      "two-wheeler":"sc341",
      "four-wheeler":"sc342",
      "commercial-vehicles":"sc343",
      "automobile-related":"sc344"
    },
    "beauty-and-health":{
        "beauty-aesthetics-and-supplies":"sc13",
        "health-care":"sc14",
        "wellness":"sc538"
    },
    "business-services":{
      "advertisement-and-media-services":"sc23",
      "logistics":"sc25",
      "it-services":"sc26",
      "financial":"sc27",
      "travel":"sc28",
      "household-services":"sc30",
      "consultancy":"sc31",
      "real-estate":"sc154",
      "waste-management-and-recycling-services":"sc407"
    },
    "education":{
      "early-education":"sc17",
      "education-consultants":"sc264",
      "education-services":"sc269",
      "k-12-education":"sc18",
      "coaching-and-tutoring":"sc19",
      "higher-education":"sc20",
      "vocational-training":"sc21",
      "online-education":"sc22"
    },
    "fashion":{
      "clothing":"sc40",
      "footwear":"sc41",
      "jewellery":"sc42",
      "luxury-premium":"sc43",
      "accessories":"sc44"
    },
    "food-and-beverage":{
      "food-and-beverage":"sc16",
      "cafe-and-parlors":"sc421",
      "quick-bites":"sc422", 
      "restaurant-and-night-clubs":"sc423",
      "bakery-sweets-and-ice-cream":"sc424",
      "catering-and-food-ordering":"sc425"
    },
    "home-based-business":{
      "home-based-businesses":"sc32",
      "technology-related":"sc274",
      "home-care-services":"sc275", 
      "beauty-and-fitness":"sc276",
      "home-based-food-preparation":"sc277",
      "home-based-manufacturing":"sc278",
      "home-based-tutoring":"sc279",
      "repair-services":"sc280",
      "business-services":"sc281"
    },
    "hotels-and-resorts":{
      "hotels-and-resorts":"sc15",
      "taxi-and-rental":"sc379",
      "tourism-services":"sc380"
    },
    "retail":{
      "consumer-durables-and-it":"sc34",
      "supermarkets-and-marts":"sc35",
      "books-toys-and-gifts":"sc36",
      "home-and-office":"sc38",
      "e-retail":"sc39",
      "fashion":"sc556"
    },
    "sports-fitness-and-entertainment":{
      "entertainment-and-leisure":"sc45",
      "sports-goods-and-fitness-stores":"sc37"  
    }
}

In [ ]:
# Create generic method to get lists

def createLists1(soup, category, sub_category):

  try:
    # Get "Company Names"
    for i in soup.find_all("div", class_="catlist"):
      company_names.update({i.text.strip():[]})

      try:
        # Get "Group Name"" & group names"
        for x in soup.find_all("div", class_="catlisthead"):
          company_names.update({i.text.strip():[category.replace("-"," ").capitalize(), sub_category.replace("-"," ").capitalize(), x.text.strip().capitalize()]})
      except:
        print('Error while updating Company attributes')
  except:
    print('Error while fetching Company Name')

  try:
    # Get Description
    for i in soup.find_all("div", class_="cattxtimg"):
      description.append(i.text.replace("\n","").replace("About Us:","").replace("AboutUs:","").replace("\r","").replace("\xa0","").replace("a.","").replace("About","").replace("Us:","").strip())
  except:
    print('Error while fetching Description')

  try:  
    for i in soup.find_all("div", class_="catdright pad20"):
      temp_text = i.text.strip()

      # Get Investment Sizes from different formats
      if "size" in temp_text:
        # INR 30 Lac - 50 Lac  
        if len(re.findall(r"INR\s+\d+\sLac\s+\-\s+\d+\sLac", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s+\d+\sLac\s+\-\s+\d+\sLac", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s+\d+\sLac\s+\-\s+\d+\sLac", temp_text)[0].strip().replace("INR","").replace("Lac","").split(" - ")[0])*100000)
          max_inv_sizes.append(int(re.findall(r"INR\s+\d+\sLac\s+\-\s+\d+\sLac", temp_text)[0].strip().replace("INR","").replace("Lac","").split(" - ")[1])*100000)
        
        # INR  50 Lac  - 1 Cr  
        elif len(re.findall(r"INR\s+\d+\sLac\s+\-\s\d\sCr", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s+\d+\sLac\s+\-\s\d\sCr", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s+\d+\sLac\s+\-\s\d\sCr", temp_text)[0].replace("INR","").replace("Lac","").replace("Cr","").strip().split(" - ")[0])*100000)
          max_inv_sizes.append(int(re.findall(r"INR\s+\d+\sLac\s+\-\s\d\sCr", temp_text)[0].replace("INR","").replace("Lac","").replace("Cr","").strip().split(" - ")[1])*10000000)

        # INR 50 Lac - 1 Cr
        elif len(re.findall(r"INR\s\d+\sLac\s\-\s\d+\sCr", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s\d+\sLac\s\-\s\d+\sCr", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s\d+\sLac\s\-\s\d+\sCr", temp_text)[0].strip().replace("INR","").replace("Lac","").replace("Cr","").split(" - ")[0])*100000)
          max_inv_sizes.append(int(re.findall(r"INR\s\d+\sLac\s\-\s\d+\sCr", temp_text)[0].strip().replace("INR","").replace("Lac","").replace("Cr","").split(" - ")[1])*10000000)

        # INR 50 K - 2 Lac
        elif len(re.findall(r"INR\s+\d+\s+K\s+\-\s+\d+\s+Lac", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s+\d+\s+K\s+\-\s+\d+\s+Lac", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s+\d+\s+K\s+\-\s+\d+\s+Lac", temp_text)[0].strip().replace("INR","").replace("K","").split(" - ")[0])*1000)
          max_inv_sizes.append(int(re.findall(r"INR\s+\d+\s+K\s+\-\s+\d+\s+Lac", temp_text)[0].strip().replace("INR","").replace("Lac","").split(" - ")[1])*100000)
        
        # INR 50 K - 2 Lac
        elif len(re.findall(r"INR\s\d+\sK\s+\-\s\d\sLac", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s\d+\sK\s+\-\s\d\sLac", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s\d+\sK\s+\-\s\d\sLac", temp_text)[0].strip().replace("INR","").replace("K","").split(" - ")[0])*1000)
          max_inv_sizes.append(int(re.findall(r"INR\s\d+\sK\s+\-\s\d\sLac", temp_text)[0].strip().replace("INR","").replace("Lac","").split(" - ")[1])*100000)

        # INR 1 Cr - 2 Cr
        elif len(re.findall(r"INR\s\d\sCr\s\-\s\d\sCr", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s\d\sCr\s\-\s\d\sCr", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s\d\sCr\s\-\s\d\sCr", temp_text)[0].strip().replace("INR","").replace("Cr","").split(" - ")[0])*10000000)
          max_inv_sizes.append(int(re.findall(r"INR\s\d\sCr\s\-\s\d\sCr", temp_text)[0].strip().replace("INR","").replace("Cr","").split(" - ")[1])*10000000)
        
        # INR 1 Cr - 2 Cr
        elif len(re.findall(r"INR\s+\d+\s+Cr\s+\-\s+\d+\s+Cr", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s+\d+\s+Cr\s+\-\s+\d+\s+Cr", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s+\d+\s+Cr\s+\-\s+\d+\s+Cr", temp_text)[0].strip().replace("INR","").replace("Cr","").split(" - ")[0])*10000000)
          max_inv_sizes.append(int(re.findall(r"INR\s+\d+\s+Cr\s+\-\s+\d+\s+Cr", temp_text)[0].strip().replace("INR","").replace("Cr","").split(" - ")[1])*10000000)

        # INR  10000 - 50 K
        elif len(re.findall(r"INR\s+\d+\s\-\s\d+\sK", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s+\d+\s\-\s\d+\sK", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s+\d+\s\-\s\d+\sK", temp_text)[0].strip().replace("INR","").split(" - ")[0]))
          max_inv_sizes.append(int(re.findall(r"INR\s+\d+\s\-\s\d+\sK", temp_text)[0].strip().replace("INR","").replace("K","").split(" - ")[1])*1000)
        
        # INR  10000  - 50 K
        elif len(re.findall(r"INR\s+\d+\s+\-\s\d+\sK\s", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s+\d+\s+\-\s\d+\sK\s", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s+\d+\s+\-\s\d+\sK\s", temp_text)[0].strip().replace("INR","").split(" - ")[0]))
          max_inv_sizes.append(int(re.findall(r"INR\s+\d+\s+\-\s\d+\sK\s", temp_text)[0].strip().replace("INR","").replace("K","").split(" - ")[1])*1000)
        
        # INR  10000  - 20 Lac
        elif len(re.findall(r"INR\s+\d+\s+\-\s\d+\sLac\s", temp_text)) > 0:
          inv_sizes.append(re.findall(r"INR\s+\d+\s+\-\s\d+\sLac\s", temp_text)[0].replace("INR","").strip())
          min_inv_sizes.append(int(re.findall(r"INR\s+\d+\s+\-\s\d+\sLac\s", temp_text)[0].strip().replace("INR","").split(" - ")[0]))
          max_inv_sizes.append(int(re.findall(r"INR\s+\d+\s+\-\s\d+\sLac\s", temp_text)[0].strip().replace("INR","").replace("Lac","").split(" - ")[1])*100000)
        
        # INR 0 - 0
        elif len(re.findall(r"INR\s\d\s\-\s\d", temp_text)) > 0:
          if re.findall(r"INR\s\d\s\-\s\d", temp_text)[0].strip().replace("INR","") == '0 - 0':
            inv_sizes.append(np.nan)
            min_inv_sizes.append(np.nan)
            max_inv_sizes.append(np.nan)
          else:
            inv_sizes.append(np.nan)
            min_inv_sizes.append(np.nan)
            max_inv_sizes.append(np.nan)
        
        # INR  0  - 0
        elif len(re.findall(r"INR\s+\d\s+\-\s\d", temp_text)) > 0:
          if re.findall(r"INR\s+\d\s+\-\s\d", temp_text)[0].strip().replace("INR","") == '0 - 0':
            inv_sizes.append(np.nan)
            min_inv_sizes.append(np.nan)
            max_inv_sizes.append(np.nan)
          else:
            inv_sizes.append(np.nan)
            min_inv_sizes.append(np.nan)
            max_inv_sizes.append(np.nan)

        # INR 0 - 0
        elif len(re.findall(r"INR\s+\d+\s+\-\s+\d+", temp_text)) > 0:
          if re.findall(r"INR\s+\d+\s+\-\s+\d+", temp_text)[0].strip().replace("INR","") == '0 - 0':
            inv_sizes.append(np.nan)
            min_inv_sizes.append(np.nan)
            max_inv_sizes.append(np.nan)
          else:
            inv_sizes.append(np.nan)
            min_inv_sizes.append(np.nan)
            max_inv_sizes.append(np.nan)
        else:
          inv_sizes.append(np.nan)
          min_inv_sizes.append(np.nan)
          max_inv_sizes.append(np.nan)
      else:
        inv_sizes.append(np.nan)
        min_inv_sizes.append(np.nan)
        max_inv_sizes.append(np.nan)
            
      # Get Space required 
        # Formats -
          # 500 - 1500
          # 2000 - 4000
          # 10 - 015
          # 50 - 100
          # 4000 - 10000
          # 000 - 000
          # 00 - 000
          # 10000 - 100000
          # 8000 - 20000
          # 10 - 100
          # -N/A-
          # 300 sq ft
          # 1000-1500 sq ft
          # 800 - 1500 Sq.ft
      if "Space" in temp_text:
        if "-N/A-" in temp_text:
          space_required.append(np.nan)
          min_space_required.append(np.nan)
          max_space_required.append(np.nan)
        elif "None" in temp_text:
          space_required.append(np.nan)
          min_space_required.append(np.nan)
          max_space_required.append(np.nan)
        elif "commercial" in temp_text:
          space_required.append(np.nan)
          min_space_required.append(np.nan)
          max_space_required.append(np.nan)
        elif len(re.findall(r"\d+\ssq\sft\n", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\ssq\sft\n", temp_text)[0].strip().replace("sq","").replace("ft",""))
          min_space_required.append(np.nan)
          max_space_required.append(re.findall(r"\d+\ssq\sft\n", temp_text)[0].strip().replace("sq","").replace("ft",""))
        elif len(re.findall(r"\d+\-\d+\ssq\sft\n", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\-\d+\ssq\sft\n", temp_text)[0].strip().replace("sq","").replace("ft",""))
          min_space_required.append(re.findall(r"\d+\-\d+\ssq\sft\n", temp_text)[0].strip().replace("sq","").replace("ft","").split("-")[0])
          max_space_required.append(re.findall(r"\d+\-\d+\ssq\sft\n", temp_text)[0].strip().replace("sq","").replace("ft","").split("-")[1])
        # 500 - 500
        elif len(re.findall(r"\d+\s\-\s\d+", temp_text)) > 0:
          if len(re.findall(r"\d+\s\-\s\d+", temp_text)) == 1:
            space_required.append(re.findall(r"\d+\s\-\s\d+", temp_text)[0].strip())
            min_space_required.append(re.findall(r"\d+\s\-\s\d+", temp_text)[0].strip().split("-")[0])
            max_space_required.append(re.findall(r"\d+\s\-\s\d+", temp_text)[0].strip().split("-")[1])
          elif len(re.findall(r"\d+\s+\-\s\d+", temp_text)) == 2:
            space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[1].strip())
            min_space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[1].strip().replace("sq","").replace("ft","").split("-")[0])
            max_space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[1].strip().replace("sq","").replace("ft","").split("-")[1])
          else:
            space_required.append(np.nan)
            min_space_required.append(np.nan)
            max_space_required.append(np.nan)
        # 1200  - 2500
        elif len(re.findall(r"\d+\s+\-\s\d+", temp_text)) > 0:
          if len(re.findall(r"\d+\s+\-\s\d+", temp_text)) == 1:
            space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[0].strip())
            min_space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[0].strip().split("-")[0])
            max_space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[0].strip().split("-")[1])
          elif len(re.findall(r"\d+\s+\-\s\d+", temp_text)) == 2:
            space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[1].strip())
            min_space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[1].strip().replace("sq","").replace("ft","").split("-")[0])
            max_space_required.append(re.findall(r"\d+\s+\-\s\d+", temp_text)[1].strip().replace("sq","").replace("ft","").split("-")[1])
          else:
            space_required.append(np.nan)
            min_space_required.append(np.nan)
            max_space_required.append(np.nan)
        # 1000 Sq. ft. to 1800 Sq. ft.
        elif len(re.findall(r"\d+\sSq\.\sft\.\sto\s\d+\sSq\.\sft\.", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\sSq\.\sft\.\sto\s\d+\sSq\.\sft\.", temp_text)[0].strip().replace("Sq.","").replace("to","-").replace("ft.",""))
          min_space_required.append(re.findall(r"\d+\sSq\.\sft\.\sto\s\d+\sSq\.\sft\.", temp_text)[0].strip().replace("Sq.","").replace("to","-").replace("ft.","").split("-")[0])
          max_space_required.append(re.findall(r"\d+\sSq\.\sft\.\sto\s\d+\sSq\.\sft\.", temp_text)[0].strip().replace("Sq.","").replace("to","-").replace("ft.","").split("-")[1])
        # 100 sft
        elif len(re.findall(r"\d+\ssft", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\ssft", temp_text)[0].strip().replace("sft",""))
          min_space_required.append(np.nan)
          max_space_required.append(re.findall(r"\d+\ssft", temp_text)[0].strip().replace("sft",""))
        # 1500sqft
        elif len(re.findall(r"\d+sqft", temp_text)) > 0:
          space_required.append(re.findall(r"\d+sqft", temp_text)[0].strip().replace("sqft",""))
          min_space_required.append(np.nan)
          max_space_required.append(re.findall(r"\d+sqft", temp_text)[0].strip().replace("sqft",""))
        # 500 to 1000 sq. ft. carpet area with 20 ft. frontage.
        elif len(re.findall(r"\d+\sto\s\d+\ssq\.\sft\.", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\sto\s\d+\ssq\.\sft\.", temp_text)[0].strip().replace("sq.","").replace("ft.",""))
          min_space_required.append(re.findall(r"\d+\sto\s\d+\ssq\.\sft\.", temp_text)[0].strip().replace("sq.","").replace("ft.","").split("to")[0].strip())
          max_space_required.append(re.findall(r"\d+\sto\s\d+\ssq\.\sft\.", temp_text)[0].strip().replace("sq.","").replace("ft.","").split("to")[1].strip())
        # 400-600 Sq. Ft.
        elif len(re.findall(r"\d+\-\d+\sSq\.\sFt\.", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\-\d+\sSq\.\sFt\.", temp_text)[0].strip().replace("Sq.","").replace("Ft.",""))
          min_space_required.append(re.findall(r"\d+\-\d+\sSq\.\sFt\.", temp_text)[0].strip().replace("Sq.","").replace("Ft.","").split("-")[0])
          max_space_required.append(re.findall(r"\d+\-\d+\sSq\.\sFt\.", temp_text)[0].strip().replace("Sq.","").replace("Ft.","").split("-")[1])
        # 400 square feet omwards
        elif len(re.findall(r"\d+\ssquare feet", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\ssquare feet", temp_text)[0].strip().replace("square","").replace("feet",""))
          min_space_required.append(np.nan)
          max_space_required.append(re.findall(r"\d+\ssquare feet", temp_text)[0].strip().replace("square","").replace("feet",""))
        # 150-300 sq feet
        elif len(re.findall(r"\d+\-\d+\ssq\sfeet", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\-\d+\ssq\sfeet", temp_text)[0].strip().replace("sq","-").replace("feet",""))
          min_space_required.append(re.findall(r"\d+\-\d+\ssq\sfeet", temp_text)[0].strip().replace("sq","-").replace("feet","").split("-")[0])
          max_space_required.append(re.findall(r"\d+\-\d+\ssq\sfeet", temp_text)[0].strip().replace("sq","-").replace("feet","").split("-")[1])
        elif len(re.findall(r"\d+\s\-\s\d+\sSq\.ft\n", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\s\-\s\d+\sSq\.ft\n", temp_text)[0].strip().replace("Sq.","").replace("ft",""))
          min_space_required.append(re.findall(r"\d+\s\-\s\d+\sSq\.ft\n", temp_text)[0].strip().replace("Sq.","").replace("ft","").split("-")[0])
          max_space_required.append(re.findall(r"\d+\s\-\s\d+\sSq\.ft\n", temp_text)[0].strip().replace("Sq.","").replace("ft","").split("-")[1])
        elif len(re.findall(r"\d+\s\-\s\d+\n", temp_text)) > 0:
          if(re.findall(r"\d+\s\-\s\d+\n", temp_text)[0].strip() == '00 - 000'):
            space_required.append(np.nan)
            min_space_required.append(np.nan)
            max_space_required.append(np.nan)
          elif(re.findall(r"\d+\s\-\s\d+\n", temp_text)[0].strip() == '000 - 000'):
            space_required.append(np.nan)
            min_space_required.append(np.nan)
            max_space_required.append(np.nan)
          else:
            space_required.append(re.findall(r"\d+\s\-\s\d+\n", temp_text)[0].strip())
            min_space_required.append(re.findall(r"\d+\s\-\s\d+\n", temp_text)[0].strip().split("-")[0]) 
            max_space_required.append(re.findall(r"\d+\s\-\s\d+\n", temp_text)[0].strip().split("-")[1])  
        # 100
        elif len(re.findall(r"\d+\n", temp_text)) > 0:
          space_required.append(re.findall(r"\d+\n", temp_text)[0].strip())
          min_space_required.append(np.nan)
          max_space_required.append(re.findall(r"\d+\n", temp_text)[0].strip())
        else:
          space_required.append(np.nan)
          min_space_required.append(np.nan)
          max_space_required.append(np.nan)
      else:
        space_required.append(np.nan)
        min_space_required.append(np.nan)
        max_space_required.append(np.nan)

      # Get Outlets count
      if "Less" in temp_text:
        outlets.append("Less than 10")
      elif "More" in temp_text:
        outlets.append("More than 10000")
      elif "None" in temp_text:
        outlets.append(np.nan)
      elif "No" in temp_text:
        outlets.append(np.nan)
      elif len(re.findall(r"\d+-\d+\n", temp_text)) > 0:
        outlets.append(re.findall(r"\d+-\d+\n", temp_text)[0].strip())
      elif len(re.findall(r"\d+\n", temp_text)) > 0:
        outlets.append(re.findall(r"\d+\n", temp_text)[0].strip())
      elif len(re.findall(r"\d\n", temp_text)) > 0:
        if re.findall(r"\d\n", temp_text)[0].strip() == '0':
          outlets.append(np.nan)
        else:
          outlets.append(re.findall(r"\d\n", temp_text)[0].strip())
      elif len(re.findall(r"\d+\n", temp_text)) > 0:
        outlets.append(re.findall(r"\d+\n", temp_text)[0].strip())
      elif len(re.findall(r"\d+\+\n", temp_text)) > 0:
        outlets.append(re.findall(r"\d+\+\n", temp_text)[0].strip())
      else:
        outlets.append(np.nan)

      # Get Headquarters location
      if "\n\nHeadquarter\n" in temp_text:
        headquarters.append(temp_text.split("\n\nHeadquarter\n")[1].strip().capitalize())
      else:
        headquarters.append(np.nan)

      # Get Franchise Type
      if "\nFranchise Type" in temp_text:
        if(len(temp_text.split("\nFranchise Type"))) > 0:
          if(len(temp_text.split("\nFranchise Type")[1])) > 0:
            franchiseType.append(temp_text.split("\nFranchise Type")[1].strip().split('   ')[0].strip())
          else:
            franchiseType.append(np.nan)
        else:
          franchiseType.append(np.nan)
      else:
        franchiseType.append(np.nan)
  except:
    print('Error fetching main details')

  try:
    for i in soup.find_all("div", class_="commanmbs"):
      temp_text = i.text.strip()
      if "Establishment year\n" in temp_text:
        establishment_year.append(temp_text.split("Establishment year\n")[1].strip().split('   ')[0].strip())
      # else:
      #   establishment_year.append(np.nan)

      # Franchising Launch Date or Dealership Launch Date
      if "\n\nFranchising Launch Date\n" in temp_text:
        franchising_launch_date.append(temp_text.split("\n\nFranchising Launch Date\n")[1].strip())
      elif "\n\nDealership Launch Date\n" in temp_text:
        franchising_launch_date.append(temp_text.split("\n\nDealership Launch Date\n")[1].strip())
      else:
        franchising_launch_date.append(np.nan)
  except:
    print('Error while fetching data for Establishment year & Franchising Launch Date')
    
  print('Lists created')

In [ ]:
# Get data from website 

# Initialize Variables

pageCount_dict = {}

company_names = {} # this will 1st add company name and then category name, sub category & group name

description = []

inv_sizes = []
max_inv_sizes = []
min_inv_sizes = []

space_required = []
max_space_required = []
min_space_required = []

outlets = []
headquarters = []
franchiseType = []

establishment_year = []
franchising_launch_date = []

try:
  for category_name, category_value in categories_dict.items():
    for sub_category_name, sub_category_value in categories_dict[category_name].items():
      page_url = "https://www.franchiseindia.com/business-opportunities/"+sub_category_name+"."+sub_category_value
      page = requests.get(page_url)
      soup = BeautifulSoup(page.text)

      pageCount = []
      
      try:
        if len(soup.find_all("ul", class_="pagination")) > 0:
          for i in soup.find_all("ul", class_="pagination")[0].text.strip().split('\n'):
            if i.isdigit():
              pageCount.append(int(i))
        else:
          pageCount.append(1) # this defaults to one page
      except:
        print('Unable to get pagination count')

      pageCount_dict.update({sub_category_name:max(pageCount)})

      max_page_count = pageCount_dict.get(sub_category_name)

      try:
        for i in range(1, (int(max_page_count)+1)):
          page_url1 = "https://www.franchiseindia.com/business-opportunities/"+sub_category_name+"."+sub_category_value+"?sortby=&page="+str(i)
          page1 = requests.get(page_url1)
          soup1 = BeautifulSoup(page1.text)
          createLists1(soup1, category_name, sub_category_name)
      except:
        print('Error while processing page')
except:
  print('Error while looping the main records')

Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists created
Lists 

In [ ]:
company_names_df = pd.DataFrame.from_dict(company_names, orient='index')

In [ ]:
company_names_df['Company_Name_temp'] = company_names_df.index
company_names_df.reset_index(inplace = True, drop = True)

In [ ]:
franchise_attr_data_dict = {
    "Description":description,
    "Investment Size":inv_sizes,
    "Min Investment Size":min_inv_sizes,
    "Max Investment Size":max_inv_sizes,
    "Space Required":space_required,
    "Min Space Required":min_space_required,
    "Max Space Required":max_space_required,
    "Franchise Outlets":outlets,
    "Headquarters":headquarters,
    "Franchise Type":franchiseType,
    "Establishment year":establishment_year,
    "Franchising Launch Date":franchising_launch_date
  }

In [ ]:
franchise_attr_df = pd.DataFrame(franchise_attr_data_dict)

In [ ]:
franchise_data_df = pd.concat([company_names_df, franchise_attr_df], axis=1, join='inner')

In [ ]:
# Reset Index on DataFrame
franchise_data_df.reset_index(drop=True, inplace=True)

In [ ]:
# Rename column names 

franchise_data_df.columns = [
    'Category', 'Sub Category', 'Services', 'Company Name',
    'Description', 
    'Investment Size', 'Min Investment Size', 'Max Investment Size', 
    'Space Required', 'Min Space Required', 'Max Space Required', 
    'Franchise Outlets',
    'Headquarters', 'Franchise Type', 'Establishment year',
    'Franchising Launch Date'
  ]

In [ ]:
# Drop columns
franchise_data_df.drop(['Investment Size', 'Space Required'], axis=1, inplace = True) # changes applied

In [ ]:
# Get dtypes from DataFrame
franchise_data_df.dtypes

Category                    object
Sub Category                object
Services                    object
Company Name                object
Description                 object
Min Investment Size        float64
Max Investment Size        float64
Min Space Required          object
Max Space Required          object
Franchise Outlets           object
Headquarters                object
Franchise Type              object
Establishment year          object
Franchising Launch Date     object
dtype: object

In [ ]:
# Get Shape from DataFrame
franchise_data_df.shape

# Get Columns from DataFrame
print(end='\n\n')
franchise_data_df.columns

# Get RangeIndex from DataFrame
print(end='\n\n')
franchise_data_df.index

# Get Descriptive statistics of the data frame

# Get details for Object columns
# result_table3.describe(include = np.object_)

# Descriptive statistics of all columns
print(end='\n\n')
franchise_data_df.describe(include = 'all')

# Same as # Get details for Numerical columns
# result_table3.describe(include = np.number)

print(end='\n\n')
franchise_data_df.describe()

# Get information from DataFrame
print(end='\n\n')
franchise_data_df.info()

(650, 14)

Index(['Category', 'Sub Category', 'Services', 'Company Name', 'Description',
       'Min Investment Size', 'Max Investment Size', 'Min Space Required',
       'Max Space Required', 'Franchise Outlets', 'Headquarters',
       'Franchise Type', 'Establishment year', 'Franchising Launch Date'],
      dtype='object')

RangeIndex(start=0, stop=650, step=1)

,Category,Sub Category,Services,Company Name,Description,Min Investment Size,Max Investment Size,Min Space Required,Max Space Required,Franchise Outlets,Headquarters,Franchise Type,Establishment year,Franchising Launch Date
count,650,650,650,650,650,6.460000e+02,6.460000e+02,586,600,632,648,629,650,650
unique,10,48,74,650,584,NaN,NaN,59,81,25,157,4,60,37
top,Food and beverage,Food and beverage,Others food service,Bharat Cycles,5 Store Photos,NaN,NaN,200,1000,Less than 10,New delhi,Unit,2018,2022
freq,246,71,71,1,55,NaN,NaN,62,58,366,74,495,53,174
mean,NaN,NaN,NaN,NaN,NaN,2.635511e+06,5.421440e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,5.832907e+06,1.280821e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,1.000000e+04,5.000000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,5.000000e+05,1.000000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,1.000000e+06,2.000000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,3.000000e+06,5.000000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Min Investment Size,Max Investment Size
count,6.460000e+02,6.460000e+02
mean,2.635511e+06,5.421440e+06
std,5.832907e+06,1.280821e+07
min,1.000000e+04,5.000000e+04
25%,5.000000e+05,1.000000e+06
50%,1.000000e+06,2.000000e+06
75%,3.000000e+06,5.000000e+06
max,5.000000e+07,1.000000e+08




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Category                 650 non-null    object 
 1   Sub Category             650 non-null    object 
 2   Services                 650 non-null    object 
 3   Company Name             650 non-null    object 
 4   Description              650 non-null    object 
 5   Min Investment Size      646 non-null    float64
 6   Max Investment Size      646 non-null    float64
 7   Min Space Required       586 non-null    object 
 8   Max Space Required       600 non-null    object 
 9   Franchise Outlets        632 non-null    object 
 10  Headquarters             648 non-null    object 
 11  Franchise Type           629 non-null    object 
 12  Establishment year       650 non-null    object 
 13  Franchising Launch Date  650 non-null    object 
dtypes: float64(2), object(12

In [ ]:
# Storing into the excel file

franchise_data_df.to_excel(r'C:\Users\z537474\Downloads\franchise_data.xlsx')
# franchise_data_df.to_excel("output.xlsx")